In [6]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [7]:
import pandas as pd
df_all = pd.read_csv('/kaggle/input/recipe-data/preprocessedDataV1.csv')

In [8]:
df = pd.DataFrame({'text': df_all['TranslatedIngredients'], 'headlines': df_all['finalResult']})
df

,text,headlines
0,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",title: Masala Karela Recipe <section> ingredie...
1,"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",title: Spicy Tomato Rice (Recipe) <section> in...
2,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",title: Ragi Semiya Upma Recipe - Ragi Millet V...
3,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",title: Gongura Chicken Curry Recipe - Andhra S...
4,"1 tablespoon chana dal, 1 tablespoon white ura...",title: Andhra Style Alam Pachadi Recipe - Adra...
...,...,...
6866,"20 बटन मशरुम,2 प्याज - काट ले,1 टमाटर - बारीक ...",title: Goan Mushroom Xacuti Recipe <section> ...
6867,"1 बड़ा चम्मच तेल,1 कप गेहूं का आटा,नमक - स्वाद ...",title: Sweet Potato & Methi Stuffed Paratha R...
6868,150 grams Spring Onion (Bulb & Greens) - chopp...,title: Ullikadala Pulusu Recipe | Spring Onion...
6869,"1 kg Chicken - medium pieces,1/2 cup Mustard o...",title: Kashmiri Style Kokur Yakhni Recipe-Chic...


In [9]:
df["text"] = "summarize: "+df["text"]

In [10]:
df.head()

,text,headlines
0,summarize: 6 Karela (Bitter Gourd/ Pavakkai) -...,title: Masala Karela Recipe <section> ingredie...
1,"summarize: 2-1 / 2 cups rice - cooked, 3 tomat...",title: Spicy Tomato Rice (Recipe) <section> in...
2,summarize: 1-1/2 cups Rice Vermicelli Noodles ...,title: Ragi Semiya Upma Recipe - Ragi Millet V...
3,"summarize: 500 grams Chicken,2 Onion - chopped...",title: Gongura Chicken Curry Recipe - Andhra S...
4,"summarize: 1 tablespoon chana dal, 1 tablespoo...",title: Andhra Style Alam Pachadi Recipe - Adra...


In [11]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [12]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [13]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [14]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [15]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [16]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
      
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [17]:
from tqdm import tqdm

model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":3,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility
}

In [18]:
T5Trainer(dataframe=df[:500], source_text="text", target_text="headlines", model_params=model_params, output_dir="outputs")

[07:27:42] [Model]: Loading t5-base...                                                             ]8;id=212512;file:///tmp/ipykernel_14/2575154273.py\2575154273.py]8;;\:]8;id=90299;file:///tmp/ipykernel_14/2575154273.py#14\14]8;;\
                                                                                                                   

/usr/local/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


[07:27:59] [Data]: Reading data...                                                                 ]8;id=380197;file:///tmp/ipykernel_14/2575154273.py\2575154273.py]8;;\:]8;id=241083;file:///tmp/ipykernel_14/2575154273.py#25\25]8;;\
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|    summarize: 6 Karela (Bitter Gourd/ Pavakkai) -      |  title: Masala Karela Recipe <section> ingredients: 6  |
|  deseeded,Salt - to taste,1 Onion - thinly sliced,3    | Karela (Bitter Gourd/ Pavakkai) - deseeded <sep> Salt -|
|  tablespoon Gram flour (besan),2 teaspoons Turmeric    |     to taste <sep> 1 Onion - thinly sliced <sep> 3     |
|    powder (Haldi),1 tablespoon Red Chilli powder,2     |     tablespoon Gram flour (besan) <sep> 2 teaspoons    |
| teaspoons Cumin seeds (Jeera),1 tablespoon Coriander   |  Turmeric powder (Haldi) <sep> 1 tablespoon Red Chilli |
|    Powder (Dhania),1 tablespoon Amchur (Dry Mango      |  powder <sep> 2 teaspoons Cumin seeds (Jeera) <sep> 1  |
|          Powder),Sunflower Oil - as required           | tablespoon Coriander Powder (Dhania) <sep> 1 tablespoon|
|                                                        |   Amchur (Dry Mango Powder) <sep> Sunflower Oil - as   |
|                                                        |   required <section> directions: To begin making the   |
|                                                        | Masala Karela Recipe,de-seed the karela and slice <sep>|
|                                                        |     Do not remove the skin as the skin has all the     |
|                                                        | nutrients <sep>  Add the karela to the pressure cooker |
|                                                        |  with 3 tablespoon of water, salt and turmeric powder  |
|                                                        | and pressure cook for three whistles <sep>  Release the|
|                                                        |   pressure immediately and open the lids <sep>  Keep   |
|                                                        |    aside <sep> Heat oil in a heavy bottomed pan or a   |
|                                                        |  kadhai <sep>  Add cumin seeds and let it sizzle <sep> |
|                                                        | Once the cumin seeds have sizzled, add onions and saute|
|                                                        | them till it turns golden brown in color <sep> Add the |
|                                                        |   karela, red chilli powder, amchur powder, coriander  |
|                                                        |   powder and besan <sep>  Stir to combine the masalas  |
|                                                        | into the karela <sep> Drizzle a little extra oil on the|
|                                                        |    top and mix again <sep>  Cover the pan and simmer   |
|                                                        |  Masala Karela stirring occasionally until everything  |
|                                                        |   comes together well <sep>  Turn off the heat <sep>   |
|                                                        |  Transfer Masala Karela into a serving bowl and serve  |
|                                                        |  <sep> Serve Masala Karela along with Panchmel Dal and |
|                                                        |    Phulka for a weekday meal with your family <sep>    |
| summarize: 2-1 / 2 cups rice - cooked, 3 tomatoes, 3   |       title: Spicy Tomato Rice (Recipe) <section>      |
|teaspoons BC Belle Bhat powder, salt - as per taste, 1  |    ingredients: 2-1 / 2 cups rice - cooked <sep>  3    |
|teaspoon chickpea lentils, 1/2 teaspoon cumin seeds, 1 

FULL Dataset: (500, 2)

...

Completed 10

[07:40:19] [Validation Completed.]                                                                 ]8;id=896481;file:///tmp/ipykernel_14/2575154273.py\2575154273.py]8;;\:]8;id=73265;file:///tmp/ipykernel_14/2575154273.py#95\95]8;;\
                                                                                                                   

[Model] Model saved @ outputs/model_files

[Validation] Generation on Validation data saved @ outputs/predictions.csv

[Logs] Logs saved @ outputs/logs.txt

In [9]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

df_all = pd.read_csv('./predictionsV1.csv')
df_all

,Unnamed: 0,Generated Text,Actual Text
0,0,"2-1/ 2 cups rice - cooked, 3 tomatoes, 3 teaspoons BC Belle Bhat powder, salt - as per taste - as per taste - 1 teaspoon chickpea lentils, 1/2 teaspoon cumin seeds, 1/2 teaspoon white urad dal, 1/2 Teaspoon mustard, 1 green chilli, 1 dry red chilli, 2 teaspoon cashew - or peanuts, 1-1/ 2 tablespoon oil - 1/2 teaspoon asafoetida - or peanuts, 1 teaspoon cashews","title: Spicy Tomato Rice (Recipe) section> ingredients: 2-1 / 2 cups rice - cooked sep> 3 tomatoes sep> 3 teaspoons BC Belle Bhat powder sep> salt - as per taste sep> 1 teaspoon chickpea lentils sep> 1/2 teaspoon cumin seeds sep> 1 teaspoon white urad dal sep> 1/2 Teaspoon mustard sep> 1 green chilli sep> 1 dry red chilli sep> 2 teaspoon cashew - or peanuts sep> 1-1 / 2 tablespoon oil - 1/2 teaspoon asafoetida section> directions: To make tomato puliogere, first cut the tomatoes sep> Now put in a mixer grinder and puree it sep> Now heat oil in a pan sep> After the oil is hot, add chana dal, urad dal, cashew and let it cook for 10 to 20 seconds sep> After 10 to 20 seconds, add cumin seeds, mustard seeds, green chillies, dry red chillies and curry leaves sep> After 30 seconds, add tomato puree to it and mix sep> Add BC Belle Bhat powder, salt and mix it sep> Allow to cook for 7 to 8 minutes and then turn off the gas sep> Take it out in a bowl, add cooked rice and mix it sep> Serve hot sep> Serve tomato puliogre with tomato cucumber raita and papad for dinner sep>"
1,1,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 Onion - sliced,1/2 cup Carrots (Gajjar) chopped,1/3 cup Green peas (Matar),2 Green Chillies,1/4 teaspoon Asafoetida (hing),1 teaspoon Mustard seeds,1 teaspoon White Urad Dal (Split),1 teaspoon Ghee,1 sprig Curry leaves, Salt - to taste,1 teaspoon Lemon juice and 1 teaspoon Lemon juice.","title: Ragi Semiya Upma Recipe - Ragi Millet Vermicelli Breakfast section> ingredients: 1-1/2 cups Rice Vermicelli Noodles (Thin) sep> 1 Onion - sliced sep> 1/2 cup Carrots (Gajjar) - chopped sep> 1/3 cup Green peas (Matar) sep> 2 Green Chillies sep> 1/4 teaspoon Asafoetida (hing) sep> 1 teaspoon Mustard seeds sep> 1 teaspoon White Urad Dal (Split) sep> 1 teaspoon Ghee sep> 1 sprig Curry leaves sep> Salt - to taste sep> 1 teaspoon Lemon juice section> directions: To begin making the Ragi Vermicelli Recipe, first steam the ragi vermicelli in a rice cooker or a steamer for about 5-6 minutes or till it is cooked but firm sep> Keep aside this aside till later use sep> You can add a few drops of oil and mix it so that they don't stick to each other sep> Place a kadai on the heat, add the ghee or oil to it and when warm add hing and allow it to sizzle for 30 seconds sep> Then follow it up with mustard seeds, urad dal and curry leaves, and allow them to crackle sep> Saute for 1 minute or so till the urad dal is slightly browned sep> Then, add onions and fry till translucent and soft sep> Next, add the green chillies along with par boiled carrots and peas sep> Sprinkle some salt and cook for 2-3 minutes or until the vegetables are semi cooked sep> Then, add the steamed ragi vermicelli toss it together so the vegetables are all well combined sep> Switch off the heat, take the vermicelli out into a serving dish and to with lemon juice sep> Mix well and serve along with Coconut Chutney and a hot cup of coffee or tea for a wholesome breakfast sep>"
2,2,"5 Potatoes (Aloo) - halved with skin, 2 teaspoon Fennel seeds (Saunf) - crushed, 2 teaspoon Red Chilli powder, 1 teaspoon Turmeric powder (Haldi), 1/4 cup Fresh cream - or 1/2 cup milk - 4 sprig Coriander (Dhania) Leaves - finely chopped.","title: Saunf Aloo (Fennel Potato Curry) Recipe section> ingredients: 5 Potatoes (Aloo) - halved with skin sep> 2 teaspoon Fennel seeds (Saunf) - crushed sep> 2 teaspoon Red Chilli powder sep> 1 teaspoon Turmeric powder (Haldi) sep> 1/4 cup Fresh cream - or 1/2 cup milk sep> 4 sprig Coriander (Dhania) Leaves - finely chopped section> directions: To begin with Saun